In [ ]:
!pip install numpy pandas tensorflow matplotlib seaborn split-folders 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
!unzip data.zip

In [4]:
import splitfolders

splitfolders.fixed("data", output="output", seed=1337, fixed=(10), oversample=True, group_prefix=None)

Copying files: 985 files [00:00, 5820.74 files/s]
Oversampling: 3 classes [00:00, 23.88 classes/s]


In [5]:
image_size = (100,115)
batch_size = 32

train = ImageDataGenerator().flow_from_directory('output/train', target_size=image_size, class_mode='categorical', batch_size=batch_size, color_mode='grayscale')
val = ImageDataGenerator().flow_from_directory('output/val', target_size=image_size, class_mode='categorical', batch_size=batch_size, color_mode='grayscale')

Found 1596 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


In [6]:
train.class_indices

{'down': 0, 'right': 1, 'up': 2}

In [7]:
train.image_shape

(100, 115, 1)

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [9]:
model = Sequential()

model.add(Conv2D(128, 3, activation='relu', input_shape = train.image_shape))
model.add(MaxPool2D(2))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPool2D(2))
model.add(Conv2D(128, 3, activation='relu'))
model.add(MaxPool2D(2))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPool2D(2))

model.add(Conv2D(128, 3, activation='relu'))
model.add(MaxPool2D(2))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 113, 128)      1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 56, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 54, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 27, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 25, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 12, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 10, 64)         7

In [10]:
training = model.fit(train, validation_data=val, epochs = 100, callbacks=[early_stopping])

Epoch 1/100
50/50 [==============================] - 34s 77ms/step - loss: 1.0309 - accuracy: 0.5370 - val_loss: 0.4683 - val_accuracy: 0.6667
Epoch 2/100
50/50 [==============================] - 3s 55ms/step - loss: 0.2022 - accuracy: 0.9273 - val_loss: 0.3439 - val_accuracy: 0.9667
Epoch 3/100
50/50 [==============================] - 3s 56ms/step - loss: 0.0472 - accuracy: 0.9912 - val_loss: 0.0106 - val_accuracy: 1.0000
Epoch 4/100
50/50 [==============================] - 3s 55ms/step - loss: 0.0375 - accuracy: 0.9931 - val_loss: 0.5176 - val_accuracy: 0.9667
Epoch 5/100
50/50 [==============================] - 3s 55ms/step - loss: 0.0298 - accuracy: 0.9950 - val_loss: 0.0078 - val_accuracy: 1.0000
Epoch 6/100
50/50 [==============================] - 3s 55ms/step - loss: 0.0271 - accuracy: 0.9950 - val_loss: 3.1522 - val_accuracy: 0.9667
Epoch 7/100
50/50 [==============================] - 3s 56ms/step - loss: 0.0323 - accuracy: 0.9950 - val_loss: 2.9014 - val_accuracy: 0.9667
Epoch

In [11]:
model.save('model_file.h5')